In [2]:
from mxnet import gluon
from mxnet import ndarray as nd

In [3]:
def transform(data,label):
    return data.astype('float32')/255,label.astype('float32')

In [4]:
mnist_train = gluon.data.vision.FashionMNIST(root='../Mx/datasets/fashion-mnist/',train=True,transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(root='../Mx/datasets/fashion-mnist/',train=False,transform=transform)

/home/zyc/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/home/zyc/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision/datasets.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


### 数据读取

In [5]:
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train,batch_size,shuffle=True)
test_data = gluon.data.DataLoader(mnist_test,batch_size,shuffle=False)

In [6]:
num_inputs = 28*28
num_outputs = 10

num_hidden = 256
weight_scale = .01

W1 = nd.random_normal(shape=(num_inputs,num_hidden),scale=weight_scale)
b1 = nd.zeros(num_hidden)

W2 = nd.random_normal(shape=(num_hidden,num_outputs),scale = weight_scale)
b2 = nd.zeros(num_outputs)

params = [W1,b1,W2,b2]

for param in params:
    param.attach_grad()

### 激活函数

如果用线性操作符来构造多层神经网络，那么整个网络任然是一个线性，这是因为  
$y’ = X*W_1*W_2 = X*W_3$  

### $relu(x)=max(x,0)$

In [8]:
def relu(x):
    return nd.maximum(x,0)

### 定义模型  
就是将各个层串联起来（输入层-隐藏层[relu]-输出层）

In [18]:
def net(X):
    X = X.reshape((-1,num_inputs))
    h1 = relu(nd.dot(X,W1)+b1)
    output = nd.dot(h1,W2)+b2
    return output

### Softmax 和交叉损失函数

In [30]:
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

#### 三个轮子函数

In [31]:
def SGD(params,lr):
    for param in params:
        param[:] = param-lr*param.grad

In [32]:
def accuracy(output,label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

In [33]:
def evaluate_accuracy(data_iterator,net):
    acc = 0
    for data,label in data_iterator:
        output = net(data)
        acc += accuracy(output,label)
    return acc/len(data_iterator)

### 训练

In [34]:
from mxnet import autograd as autograd

learning_rate = 0.1

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data,label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output,label)
        loss.backward()
        SGD(params,learning_rate/batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output,label)
        
    test_acc = evaluate_accuracy(test_data,net)
    print("Epoch %d. Loss: %f, Train acc %f,Test acc %f"%\
         (epoch,train_loss/len(train_data),train_acc/len(train_data),
         test_acc/len(test_data)))
        

Epoch 0. Loss: 0.287572, Train acc 0.895163,Test acc 0.022224
Epoch 1. Loss: 0.284781, Train acc 0.895678,Test acc 0.022222
Epoch 2. Loss: 0.281756, Train acc 0.897047,Test acc 0.022273
Epoch 3. Loss: 0.279007, Train acc 0.898814,Test acc 0.022258
Epoch 4. Loss: 0.276812, Train acc 0.899934,Test acc 0.022334


# 第二课，30分钟，上面测试集acc有问题，明天在注意检查